In [1]:
!pip install tensorflow


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install tensorflow.compat.v1

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow.compat.v1 (from versions: none)
ERROR: No matching distribution found for tensorflow.compat.v1

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import os
from tensor2tensor.utils import registry
from tensor2tensor.utils import trainer_lib
from tensor2tensor import problems
from tensor2tensor.models import transformer
import tensorflow as tf

# Setup directories
data_dir = os.path.expanduser("~/t2t/data")
tmp_dir = os.path.expanduser("~/t2t/tmp")
train_dir = os.path.expanduser("~/t2t/train")
checkpoint_dir = os.path.expanduser("~/t2t/checkpoints")

os.makedirs(data_dir, exist_ok=True)
os.makedirs(tmp_dir, exist_ok=True)
os.makedirs(train_dir, exist_ok=True)
os.makedirs(checkpoint_dir, exist_ok=True)

# Problem and HParams
problem_name = "translate_enfr_wmt32k"
model_name = "transformer"
hparams_set = "transformer_base"

# Fetch the problem
problem = problems.problem(problem_name)

# Generate data
from tensor2tensor.utils import data_reader

t2t_datagen_cmd = f"t2t-datagen --data_dir={data_dir} --tmp_dir={tmp_dir} --problem={problem_name}"
os.system(t2t_datagen_cmd)

# Create HParams
hparams = trainer_lib.create_hparams(hparams_set, data_dir=data_dir, problem_name=problem_name)

# Create the model
model = registry.model(model_name)(hparams, tf.estimator.ModeKeys.TRAIN)

# Create the estimator
run_config = trainer_lib.create_run_config(model_name, model_dir=train_dir)
estimator = trainer_lib.create_estimator(model_name, hparams, run_config, decode_hparams=None)

# Train the model
train_input_fn = problem.make_estimator_input_fn(tf.estimator.ModeKeys.TRAIN, data_dir)
estimator.train(train_input_fn, steps=1000)

# Decode
decode_hparams = trainer_lib.create_decode_hparams(decode_hparams=None)
decode_input_fn = problem.make_estimator_input_fn(tf.estimator.ModeKeys.PREDICT, data_dir)
predictions = estimator.predict(decode_input_fn)

# Print some predictions
for prediction in predictions:
    print(prediction['outputs'])
    break


ImportError: cannot import name 'estimator' from 'tensorflow.compat.v1' (c:\Users\steve\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\_api\v2\compat\v1\__init__.py)